In [1]:
!pip install pyyaml h5py #to save models in HDF5 format (if needed)

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION) #print tensor flow version


2.6.0


In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.metrics import auc,roc_auc_score,roc_curve,precision_score,recall_score,f1_score
import time as timer
from sklearn.inspection import permutation_importance

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
import tempfile

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [5]:
#load dataset from kaggle or elsewhere
url = "https://gist.githubusercontent.com/aishwarya8615/d2107f828d3f904839cbcb7eaa85bd04/raw/cec0340503d82d270821e03254993b6dede60afb/healthcare-dataset-stroke-data.csv"
data = pd.read_csv(url)
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
data.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [7]:
#Again like we did before for XGBoost we can drop the null values
data=data.dropna()
data.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [8]:
data['age'] = data['age'].astype(np.float32)
data['avg_glucose_level'] = data['avg_glucose_level'].astype(np.float32)
data['bmi'] = data['bmi'].astype(np.float32)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [9]:
df=data[['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi','stroke']]
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
0,67.0,0,1,228.690002,36.599998,1
2,80.0,0,1,105.919998,32.500000,1
3,49.0,0,0,171.229996,34.400002,1
4,79.0,1,0,174.119995,24.000000,1
5,81.0,0,0,186.210007,29.000000,1


In [10]:
# check data type of each column make sure float64 are now float32
df.dtypes

age                  float32
hypertension           int64
heart_disease          int64
avg_glucose_level    float32
bmi                  float32
stroke                 int64
dtype: object

In [11]:
data['age']

0       67.0
2       80.0
3       49.0
4       79.0
5       81.0
        ... 
5104    13.0
5106    81.0
5107    35.0
5108    51.0
5109    44.0
Name: age, Length: 4909, dtype: float32

In [12]:
data['bmi']

0       36.599998
2       32.500000
3       34.400002
4       24.000000
5       29.000000
          ...    
5104    18.600000
5106    40.000000
5107    30.600000
5108    25.600000
5109    26.200001
Name: bmi, Length: 4909, dtype: float32

In [13]:
data['avg_glucose_level']

0       228.690002
2       105.919998
3       171.229996
4       174.119995
5       186.210007
           ...    
5104    103.080002
5106    125.199997
5107     82.989998
5108    166.289993
5109     85.279999
Name: avg_glucose_level, Length: 4909, dtype: float32

In [14]:
# we then encode non numerical columns as we did before with XGBoost
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['gender']=le.fit_transform(data['gender'])
df['ever_married']=le.fit_transform(data['ever_married'])
df['work_type']=le.fit_transform(data['work_type'])
df['Residence_type']=le.fit_transform(data['Residence_type'])
df['smoking_status']=le.fit_transform(data['smoking_status'])
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender,ever_married,work_type,Residence_type,smoking_status
0,67.0,0,1,228.690002,36.599998,1,1,1,2,1,1
2,80.0,0,1,105.919998,32.500000,1,1,1,2,0,2
3,49.0,0,0,171.229996,34.400002,1,0,1,2,1,3
4,79.0,1,0,174.119995,24.000000,1,0,1,3,0,2
5,81.0,0,0,186.210007,29.000000,1,1,1,2,1,1


In [15]:
data.shape,df.shape

((4909, 12), (4909, 11))

In [16]:
y=df['stroke']
x=df.drop('stroke',axis=1)
x.shape,y.shape

((4909, 10), (4909,))

In [17]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(x,y,test_size=0.2)

In [18]:
 # Define a simple sequential model
def create_model():
      ann_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(1, 10)),
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(25, activation='relu'),
      
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])
     # ann_model=tf.keras.Sequential()

      # ann_model.add(tf.keras.layers.Dense(units=25,activation='relu'))

      # ann_model.add(tf.keras.layers.Dense(units=25,activation='relu'))

      # ann_model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

      ann_model.compile('adam','binary_crossentropy',metrics=['accuracy'])
      return ann_model

In [19]:
# nn_model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(1, 10)),
#   tf.keras.layers.Dense(25, activation='relu'),
#   tf.keras.layers.Dense(25, activation='relu'),
  
#   tf.keras.layers.Dense(1, activation='sigmoid')
# ])
nn_model = create_model()
nn_model.compile('adam','binary_crossentropy',metrics=['accuracy'])



#tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)
result=nn_model.fit(x_train,y_train, epochs=11)

Epoch 1/11
123/123 [==============================] - 1s 2ms/step - loss: 0.4638 - accuracy: 0.9119
Epoch 2/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1777 - accuracy: 0.9552
Epoch 3/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1781 - accuracy: 0.9559
Epoch 4/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1749 - accuracy: 0.9549
Epoch 5/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1702 - accuracy: 0.9554
Epoch 6/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1685 - accuracy: 0.9559
Epoch 7/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1712 - accuracy: 0.9544
Epoch 8/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1717 - accuracy: 0.9549
Epoch 9/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1679 - accuracy: 0.9549
Epoch 10/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1696 - accuracy: 0.9544

In [20]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 25)                275       
_________________________________________________________________
dense_1 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 951
Trainable params: 951
Non-trainable params: 0
_________________________________________________________________


In [21]:
from sklearn.metrics import confusion_matrix
y_pred=[]
for i in nn_model.predict(x_test):
    if i>0.5:
        y_pred.append(1)
    if i<0.5:
        y_pred.append(0)
confusion_matrix(y_test,y_pred)

array([[942,   0],
       [ 40,   0]])

In [22]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9592668024439919

In [23]:
# Create a Callback During training
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [24]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [25]:
# Train the model with the new callback
  # Pass callback to training
nn_model.fit(x_train,y_train,epochs=11, validation_data=(x_test, y_test),callbacks=[cp_callback])
# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/11
123/123 [==============================] - 1s 4ms/step - loss: 0.1683 - accuracy: 0.9559 - val_loss: 0.1628 - val_accuracy: 0.9501

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1672 - accuracy: 0.9547 - val_loss: 0.1637 - val_accuracy: 0.9532

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1690 - accuracy: 0.9549 - val_loss: 0.1515 - val_accuracy: 0.9593

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1659 - accuracy: 0.9554 - val_loss: 0.1699 - val_accuracy: 0.9470

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1605 - accuracy: 0.9562 - val_loss: 0.1553 - val_accuracy: 0.9542

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/11
123/123 [=========================

In [26]:
os.listdir(checkpoint_dir)


['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

In [27]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))


31/31 - 0s - loss: 0.7582 - accuracy: 0.9593
Untrained model, accuracy: 95.93%


In [28]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


31/31 - 0s - loss: 0.1571 - accuracy: 0.9593
Restored model, accuracy: 95.93%


In [29]:
# Create and train a new model instance.
model = create_model()
model.fit(x_train, y_train, epochs=11)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/stroke_model')

Epoch 1/11
123/123 [==============================] - 1s 2ms/step - loss: 0.4357 - accuracy: 0.9399
Epoch 2/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1888 - accuracy: 0.9539
Epoch 3/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1732 - accuracy: 0.9549
Epoch 4/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1794 - accuracy: 0.9567
Epoch 5/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1772 - accuracy: 0.9562
Epoch 6/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1768 - accuracy: 0.9534
Epoch 7/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1754 - accuracy: 0.9552
Epoch 8/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1709 - accuracy: 0.9557
Epoch 9/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1635 - accuracy: 0.9570
Epoch 10/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1650 - accuracy: 0.9562

In [30]:
ls saved_model

stroke_model/


In [31]:
ls saved_model/stroke_model

assets/  keras_metadata.pb  saved_model.pb  variables/


In [32]:
# Convert .pb into  .tflite
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
assert float(tf.__version__[:3]) >= 2.3


In [33]:
new_stroke_model = tf.keras.models.load_model('saved_model/stroke_model')

# Check its architecture
new_stroke_model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 25)                275       
_________________________________________________________________
dense_7 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 26        
Total params: 951
Trainable params: 951
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Evaluate the restored model
loss, acc = new_stroke_model.evaluate(x_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_stroke_model.predict(x_test).shape)


31/31 - 0s - loss: 0.1647 - accuracy: 0.9562
Restored model, accuracy: 95.62%
(982, 1)


In [35]:
new_stroke_model.fit(x_train, y_train, epochs=11)

Epoch 1/11
123/123 [==============================] - 1s 2ms/step - loss: 0.1683 - accuracy: 0.9544
Epoch 2/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1662 - accuracy: 0.9552
Epoch 3/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1653 - accuracy: 0.9567
Epoch 4/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1585 - accuracy: 0.9565
Epoch 5/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1699 - accuracy: 0.9575
Epoch 6/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1746 - accuracy: 0.9539
Epoch 7/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1655 - accuracy: 0.9552
Epoch 8/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1615 - accuracy: 0.9547
Epoch 9/11
123/123 [==============================] - 0s 3ms/step - loss: 0.1612 - accuracy: 0.9559
Epoch 10/11
123/123 [==============================] - 0s 2ms/step - loss: 0.1690 - accuracy: 0.9562

In [36]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_stroke_model)

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
]


fb_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpqrvbge2w/assets


In [37]:
with open('stroke_tflite_models/stroke_model.tflite', 'wb') as f: 
  f.write(fb_model) 

In [38]:
ls -a


./   .config/             rune/         saved_model/           training_1/
../  .ipynb_checkpoints/  sample_data/  stroke_tflite_models/


In [39]:
pwd

'/content'

In [40]:
# path_to_zip_file = "rune/rune.x86_64-unknown-linux-gnu.zip"
# directory_to_extract_to = "rune/"

In [41]:
# import zipfile
# with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

In [42]:
#mv rune ~/.cargo/bin when in workspace
